In [14]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

iris = load_iris()
X = iris['data'] # (150, 4)
y = iris['target'] # (150, ), 각 원소는 출력에서 정답의 인덱스 번호를 의미하는듯함
names = iris['target_names'] # 3종류
feature_names = iris['feature_names'] # 4개의 feature

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [18]:
std_scaler = StandardScaler()
std_scaler.fit(X_train)
X_train_tensor = torch.from_numpy(std_scaler.transform(X_train)).float()
X_test_tensor = torch.from_numpy(std_scaler.transform(X_test)).float()
y_train_tensor = torch.from_numpy(y_train).long()
y_test_tensor = torch.from_numpy(y_test).long()

print(X_train_tensor.shape, X_test_tensor.shape, y_train_tensor.shape, y_test_tensor.shape)

torch.Size([120, 4]) torch.Size([30, 4]) torch.Size([120]) torch.Size([30])


In [19]:
nb_epochs = 1000
minibatch_size = 120

In [29]:
class FunModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.linear_layers = nn.Sequential(
            nn.Linear(input_dim, 100),
            nn.LeakyReLU(0.1),
            nn.Linear(100, 20),
            nn.LeakyReLU(0.1),
            nn.Linear(20, 5),
            nn.LeakyReLU(0.1),
            nn.Linear(5, output_dim),
            nn.LogSoftmax(dim=-1) # 최종 출력은 3이 되므로 dim=-1로 적용되는 차원을 지정해준다.
        )

    def forward(self, x):
        y = self.linear_layers(x)
        return y

In [30]:
input_dim = X_train_tensor.size(-1)
output_dim = 3
print(input_dim, output_dim)
model = FunModel(input_dim, output_dim)

loss_func = nn.NLLLoss() # logsoftmax를 썼으므로 NNL Loss를 써준다.
optimizer = torch.optim.Adam(model.parameters())

4 3


In [32]:
for index in range(nb_epochs):
    indices = torch.randperm(X_train_tensor.size(0))

    x_batch_list = torch.index_select(X_train_tensor, 0, index=indices)
    y_batch_list = torch.index_select(y_train_tensor, 0, index=indices)
    x_batch_list = x_batch_list.split(minibatch_size, 0)
    y_batch_list = y_batch_list.split(minibatch_size, 0)

    epoch_loss = list()
    for x_minibatch, y_minibatch in zip(x_batch_list, y_batch_list):
        y_minibatch_pred = model(x_minibatch)
        loss = loss_func(y_minibatch_pred, y_minibatch)
        epoch_loss.append(loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (index % 100) == 0:
        print(index, sum(epoch_loss) / len(epoch_loss))

0 tensor(1.0798, grad_fn=<DivBackward0>)
100 tensor(0.2101, grad_fn=<DivBackward0>)
200 tensor(0.0447, grad_fn=<DivBackward0>)
300 tensor(0.0323, grad_fn=<DivBackward0>)
400 tensor(0.0240, grad_fn=<DivBackward0>)
500 tensor(0.0161, grad_fn=<DivBackward0>)
600 tensor(0.0094, grad_fn=<DivBackward0>)
700 tensor(0.0051, grad_fn=<DivBackward0>)
800 tensor(0.0029, grad_fn=<DivBackward0>)
900 tensor(0.0018, grad_fn=<DivBackward0>)


# 테스트셋 적용

In [33]:
model.eval()
with torch.no_grad():
    y_test_pred = model(X_test_tensor)
    # torch.argmax(x, dim=차원), 특정 차원의 데이터 중 가장 높은 값(확률)의 index 번호를 리턴
    y_pred_list = torch.argmax(y_test_pred, dim=1) # (30, 3)에서 3개의 feature 중에 가장 높은 값(확률)의 index 번호가 저장됨([0.2, 0.5, 0.3]이면 1)

# 미니배치 사이즈 기반 예측(기존내용 복습)

In [35]:
y_pred_list = list()
x_test_batch_list = X_test_tensor.split(minibatch_size, 0)
model.eval()
with torch.no_grad():
    for x_minibatch in x_test_batch_list:
        y_test_pred = model(x_minibatch)
        # torch.Size([30, 3])
        y_test_pred = torch.argmax(y_test_pred, dim=1)
        # torch.Size([30, ])
        y_pred_list.extend(y_test_pred.squeeze().detach().tolist())

y_pred_list = torch.tensor(y_pred_list)

In [38]:
print(y_pred_list.shape, y_test_tensor.shape)

torch.Size([30]) torch.Size([30])


# Multi-Label Classfication 기본 메트릭

In [48]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

print("Confusion Matrix\n", str(confusion_matrix(y_test_tensor, y_pred_list)))
print("Precision List:\t", str(precision_score(y_test_tensor, y_pred_list, average=None)))
print("Macro Precision:\t", str(precision_score(y_test_tensor, y_pred_list, average='macro')))
print("Macro Precision Formula:", str(sum(precision_score(y_test_tensor, y_pred_list, average=None)) / 3))
print("Micro Precision:\t", str(precision_score(y_test_tensor, y_pred_list, average='micro')))

print("Recall List:\t", str(precision_score(y_test_tensor, y_pred_list, average=None)))
print("Macro Recall:\t", str(recall_score(y_test_tensor, y_pred_list, average='macro')))
print("Macro Recall:\t", str(recall_score(y_test_tensor, y_pred_list, average='micro')))

print("Macro F1 Score List:\t", str(precision_score(y_test_tensor, y_pred_list, average=None)))
print("Macro F1 Score:\t", str(f1_score(y_test_tensor, y_pred_list, average='macro')))
print("Micro F1 Score:\t", str(f1_score(y_test_tensor, y_pred_list, average='micro')))

Confusion Matrix
 [[10  0  0]
 [ 0  9  1]
 [ 0  0 10]]
Precision List:	 [1.         1.         0.90909091]
Macro Precision:	 0.9696969696969697
Macro Precision Formula: 0.9696969696969697
Micro Precision:	 0.9666666666666667
Recall List:	 [1.         1.         0.90909091]
Macro Recall:	 0.9666666666666667
Macro Recall:	 0.9666666666666667
Macro F1 Score List:	 [1.         1.         0.90909091]
Macro F1 Score:	 0.9665831244778612
Micro F1 Score:	 0.9666666666666667
